In [1]:
import simpy
from random import seed, random, expovariate, normalvariate, triangular
import numpy as np
import pandas as pd
import math

## Walkin Patients Process

In [2]:
def walkin_patient(env, name, arrival_time, routine_care, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage):
    # Simulate patient arrivals to the PSC (door in)
    yield env.timeout(arrival_time)
    #print('%s walk in at %d' % (name, env.now))
    door_in = env.now
    
    # Triage
    #print('%s Triage start at %d' % (name, env.now))
    yield env.timeout(triage_duration)
    #print('%s Triage ended at %d' % (name, env.now))
    Triage_times.append(triage_duration)
    
    # ED Evaluation
    #print('%s ED Evaluation start at %d' % (name, env.now))
    yield env.timeout(ED_eval_duration)
    #print('%s ED Evaluation ended at %d' % (name, env.now))
    ED_eval_times.append(ED_eval_duration)
    
    # Stroke identified? (90% yes, 10% no) if not (10%), then add extra time in routine care
    if random() > stroke_identified_percentage:
        with routine_care.request() as req:
            yield req
            #print('%s Routine Care at %d' % (name, env.now))
            yield env.timeout(routine_care_duration)
            #print('%s Routine Care ended at %d' % (name, env.now))
            routine_care_times.append(routine_care_duration)
    
    #### Stroke Code Activation ###
    early_send = 0
    if random() < early_send_1_percentage:
        early_send = 1
        # Ambulance Contact and Arrival
        #print('%s Ambulance contact at %d' % (name, env.now))
        yield env.timeout(ambulance_arrival_duration)
        #print('%s Ambulance arrived at %d' % (name, env.now))
        Ambulance_times.append(ambulance_arrival_duration)

        # CSC (door out)
        #print('%s Patient send to CSC at %d' % (name, env.now))
        door_out = env.now
        #print('%s DIDO:' % name, door_out - door_in)
        DIDO_times.append(door_out - door_in)
    
    ####### EARLY SEND 1 ########
    if early_send != 1:
        # CTA decision (47% yes)
        # if random() < CTA_decision_percentage:
        #print('%s CTA Scan at %d' % (name, env.now))
        yield env.timeout(CTA_duration)
        #print('%s CTA Scan ended at %d' % (name, env.now))
        CTA_times.append(CTA_duration)

        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            #print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            #print('%s Ambulance arrived at %d' % (name, env.now))
            Ambulance_times.append(ambulance_arrival_duration)

            # CSC (door out)
            #print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            #print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        # Ischemic Stroke 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            #print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            #print('%s Telestroke ended at %d' % (name, env.now))
            Telestroke_times.append(telestroke_duration)
            
            if random() < early_send_2_percentage:
                early_send = 2
                # Ambulance Contact and Arrival
                #print('%s Ambulance contact at %d' % (name, env.now))
                yield env.timeout(ambulance_arrival_duration)
                #print('%s Ambulance arrived at %d' % (name, env.now))
                Ambulance_times.append(ambulance_arrival_duration)

                # CSC (door out)
                #print('%s Patient send to CSC at %d' % (name, env.now))
                door_out = env.now
                #print('%s DIDO:' % name, door_out - door_in)
                DIDO_times.append(door_out - door_in)
                # IS DIDO times
                is_DIDO_times.append(door_out - door_in)
                
            ####### EARLY SEND 2 ########
            if early_send != 2:
                # TPA (5% gets TPA)
                consent = 0
                if random() < TPA_decision_percentage:
                    # (90% consent)
                    if random() <  TPA_consent_percentage:
                        consent = 1
                        # TPA long consent duration (90%, 10% don't take time)
                        if random() < TPA_long_consent_percentage:
                            #print('%s TPA consent at %d' % (name, env.now))
                            yield env.timeout(TPA_consent_duration)
                            #print('%s TPA consent ended at %d' % (name, env.now))
                            Long_Consent_times.append(TPA_consent_duration)

                        # pharmacist in ED or not? yes if between 8am and 6pm
                        if ((env.now%1440)/60 > 8 and (env.now%1440)/60 < 18):
                            #print('%s TPA mixing at %d' % (name, env.now))
                            yield env.timeout(TPA_mixing_duration)
                            #print('%s TPA mixing ended and injection at %d' % (name, env.now))
                            Mix_TPA_times.append(TPA_mixing_duration)
                            # door to needle time
                            door_to_needle.append(env.now - door_in)
                        else:
                            #print('%s TPA mixing at %d' % (name, env.now))
                            yield env.timeout(TPA_mixing_duration + pharmacist_arrival_duration)
                            #print('%s TPA mixing ended and injection at %d' % (name, env.now))
                            Mix_TPA_times.append(TPA_mixing_duration)
                            Pharmacist_Arrival_times.append(pharmacist_arrival_duration)
                            # door to needle time
                            door_to_needle.append(env.now - door_in)
                    else: # (10% don't consent)
                        consent = 0
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in)
                 # no need for TPA
                else:
                    # transfer decision (15% yes)
                    if random() < transfer_decision_percentage:
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in) 
                    else:
                        # ICU
                        door_out = env.now
                        #print('%s ICU out time:' % name, door_out - door_in)
                        ICU_times.append(door_out - door_in) 

                if consent:
                    ####### EARLY SEND 3 ########
                    if random() < early_send_3_percentage:
                        early_send = 3
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in)
                    
                    if early_send != 3:
                        # transfer decision (15% yes)
                        if random() < transfer_decision_percentage:
                            # Ambulance Contact and Arrival
                            #print('%s Ambulance contact at %d' % (name, env.now))
                            yield env.timeout(ambulance_arrival_duration)
                            #print('%s Ambulance arrived at %d' % (name, env.now))
                            Ambulance_times.append(ambulance_arrival_duration)

                            # CSC (door out)
                            #print('%s Patient send to CSC at %d' % (name, env.now))
                            door_out = env.now
                            #print('%s DIDO:' % name, door_out - door_in)
                            DIDO_times.append(door_out - door_in)
                            # IS DIDO times
                            is_DIDO_times.append(door_out - door_in) 
                        else:
                            # ICU
                            door_out = env.now
                            #print('%s ICU out time:' % name, door_out - door_in) 
                            ICU_times.append(door_out - door_in) 

## EMS Patients Process

In [3]:
def EMS_patient(env, name, arrival_time, routine_care, EMS_screen_percentage, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage,  TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage):
    # Simulate patient arrivals to the PSC (door in)
    yield env.timeout(arrival_time)
    #print('%s EMS arrival at %d' % (name, env.now))
    door_in = env.now
    
    # EMS screened as stroke? (83% yes, 17% no) if EMS didn't screen as stroke, then add triage, ed evaluation[, and maybe routine care]
    if random() > EMS_screen_percentage:
        # Triage
        #print('%s Triage start at %d' % (name, env.now))
        yield env.timeout(triage_duration)
        #print('%s Triage ended at %d' % (name, env.now))
        Triage_times.append(triage_duration)

        # ED Evaluation
        #print('%s ED Evaluation start at %d' % (name, env.now))
        yield env.timeout(ED_eval_duration)
        #print('%s ED Evaluation ended at %d' % (name, env.now))
        ED_eval_times.append(ED_eval_duration)

        # Stroke identified? (90% yes, 10% no) if not, then add extra time in routine care
        if random() > stroke_identified_percentage:
            with routine_care.request() as req:
                yield req
                #print('%s Routine Care at %d' % (name, env.now))
                yield env.timeout(routine_care_duration)
                #print('%s Routine Care ended at %d' % (name, env.now))
                routine_care_times.append(routine_care_duration)
        
    #### Stroke Code Activation ###
    early_send = 0
    if random() < early_send_1_percentage:
        early_send = 1
        # Ambulance Contact and Arrival
        #print('%s Ambulance contact at %d' % (name, env.now))
        yield env.timeout(ambulance_arrival_duration)
        #print('%s Ambulance arrived at %d' % (name, env.now))
        Ambulance_times.append(ambulance_arrival_duration)

        # CSC (door out)
        #print('%s Patient send to CSC at %d' % (name, env.now))
        door_out = env.now
        #print('%s DIDO:' % name, door_out - door_in)
        DIDO_times.append(door_out - door_in)
    
    ####### EARLY SEND 1 ########
    if early_send != 1:
        # CTA decision (47% yes)
        # if random() < CTA_decision_percentage:
        #print('%s CTA Scan at %d' % (name, env.now))
        yield env.timeout(CTA_duration)
        #print('%s CTA Scan ended at %d' % (name, env.now))
        CTA_times.append(CTA_duration)

        # HSvIS (15% HS)
        if random() < HS_percentage: 
            # Hemorrhagic stroke -> Ambulance Contact, Ambulance Arrival, CSC
            # Ambulance Contact and Arrival
            #print('%s Ambulance contact at %d' % (name, env.now))
            yield env.timeout(ambulance_arrival_duration)
            #print('%s Ambulance arrived at %d' % (name, env.now))
            Ambulance_times.append(ambulance_arrival_duration)

            # CSC (door out)
            #print('%s Patient send to CSC at %d' % (name, env.now))
            door_out = env.now
            #print('%s DIDO:' % name, door_out - door_in)
            DIDO_times.append(door_out - door_in)
            # HS DIDO times
            hs_DIDO_times.append(door_out - door_in) 
        # Ischemic Stroke 
        else:
            # Ischemic stroke (85%)
            # Telestroke
            #print('%s Telestroke at %d' % (name, env.now))
            yield env.timeout(telestroke_duration)
            #print('%s Telestroke ended at %d' % (name, env.now))
            Telestroke_times.append(telestroke_duration)
            
            if random() < early_send_2_percentage:
                early_send = 2
                # Ambulance Contact and Arrival
                #print('%s Ambulance contact at %d' % (name, env.now))
                yield env.timeout(ambulance_arrival_duration)
                #print('%s Ambulance arrived at %d' % (name, env.now))
                Ambulance_times.append(ambulance_arrival_duration)

                # CSC (door out)
                #print('%s Patient send to CSC at %d' % (name, env.now))
                door_out = env.now
                #print('%s DIDO:' % name, door_out - door_in)
                DIDO_times.append(door_out - door_in)
                # IS DIDO times
                is_DIDO_times.append(door_out - door_in)
                
            ####### EARLY SEND 2 ########
            if early_send != 2:
                # TPA (5% gets TPA)
                consent = 0
                if random() < TPA_decision_percentage:
                    # (90% consent)
                    if random() <  TPA_consent_percentage:
                        consent = 1
                        # TPA long consent duration (90%, 10% don't take time)
                        if random() < TPA_long_consent_percentage:
                            #print('%s TPA consent at %d' % (name, env.now))
                            yield env.timeout(TPA_consent_duration)
                            #print('%s TPA consent ended at %d' % (name, env.now))
                            Long_Consent_times.append(TPA_consent_duration)

                        # pharmacist in ED or not? yes if between 8am and 6pm
                        if ((env.now%1440)/60 > 8 and (env.now%1440)/60 < 18):
                            #print('%s TPA mixing at %d' % (name, env.now))
                            yield env.timeout(TPA_mixing_duration)
                            #print('%s TPA mixing ended and injection at %d' % (name, env.now))
                            Mix_TPA_times.append(TPA_mixing_duration)
                            # door to needle time
                            door_to_needle.append(env.now - door_in)
                        else:
                            #print('%s TPA mixing at %d' % (name, env.now))
                            yield env.timeout(TPA_mixing_duration + pharmacist_arrival_duration)
                            #print('%s TPA mixing ended and injection at %d' % (name, env.now))
                            Mix_TPA_times.append(TPA_mixing_duration)
                            Pharmacist_Arrival_times.append(pharmacist_arrival_duration)
                            # door to needle time
                            door_to_needle.append(env.now - door_in)
                    else: # (10% don't consent)
                        consent = 0
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in)
                # no need for TPA
                else:
                    # transfer decision (15% yes)
                    if random() < transfer_decision_percentage:
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in) 
                    else:
                        # ICU
                        door_out = env.now
                        #print('%s ICU out time:' % name, door_out - door_in)
                        ICU_times.append(door_out - door_in) 

                if consent:
                    ####### EARLY SEND 3 ########
                    if random() < early_send_3_percentage:
                        early_send = 3
                        # Ambulance Contact and Arrival
                        #print('%s Ambulance contact at %d' % (name, env.now))
                        yield env.timeout(ambulance_arrival_duration)
                        #print('%s Ambulance arrived at %d' % (name, env.now))
                        Ambulance_times.append(ambulance_arrival_duration)

                        # CSC (door out)
                        #print('%s Patient send to CSC at %d' % (name, env.now))
                        door_out = env.now
                        #print('%s DIDO:' % name, door_out - door_in)
                        DIDO_times.append(door_out - door_in)
                        # IS DIDO times
                        is_DIDO_times.append(door_out - door_in)
                    
                    if early_send != 3:
                        # transfer decision (15% yes)
                        if random() < transfer_decision_percentage:
                            # Ambulance Contact and Arrival
                            #print('%s Ambulance contact at %d' % (name, env.now))
                            yield env.timeout(ambulance_arrival_duration)
                            #print('%s Ambulance arrived at %d' % (name, env.now))
                            Ambulance_times.append(ambulance_arrival_duration)

                            # CSC (door out)
                            #print('%s Patient send to CSC at %d' % (name, env.now))
                            door_out = env.now
                            #print('%s DIDO:' % name, door_out - door_in)
                            DIDO_times.append(door_out - door_in)
                            # IS DIDO times
                            is_DIDO_times.append(door_out - door_in) 
                        else:
                            # ICU
                            door_out = env.now
                            #print('%s ICU out time:' % name, door_out - door_in)
                            ICU_times.append(door_out - door_in) 

# General ED Patients Process

In [4]:
def ED_patient(env, name, arrival_time, routine_care_duration):
    yield env.timeout(arrival_time)
    with routine_care.request() as req:
        yield req
#         print('%s Routine Care at %d' % (name, env.now))
        yield env.timeout(routine_care_duration)
#         print('%s Routine Care ended at %d' % (name, env.now))

## Data Input

In [5]:
path = './'
data = pd.read_csv(path + 'simulation_data.csv')
print('Previewing simulation data...')
data

Previewing simulation data...


,process,count,mean(mins),triangular,percentage,distribution/type,Notes
0,arrival_rate,NaN,2160.00,NaN,NaN,poisson,1.5 days on average per stroke patient
1,EMS_arrivals_percentage,NaN,NaN,NaN,0.70,percentage,percentage of arrivals by EMS (not walk-in)
2,EMS_screen_percentage,NaN,NaN,NaN,0.83,percentage,NaN
3,triage_duration,NaN,6.00,NaN,NaN,exponential,NaN
4,ED_eval_duration,NaN,7.80,NaN,NaN,exponential,NaN
5,stroke_identified_percentage,NaN,NaN,NaN,0.90,percentage,NaN
6,routine_care_duration,NaN,90.00,NaN,NaN,exponential,NaN
7,CT_duration,NaN,28.33,NaN,NaN,exponential,NaN
8,HS_percentage,NaN,NaN,NaN,0.15,percentage,NaN
9,ambulance_arrival_duration,NaN,20.00,NaN,NaN,exponential,NaN


In [6]:
# ED arrival data for routine care
ED_arrival_data = pd.read_excel('./LFH ED arrival rate 2019.xlsx')
print('Previewing ED arrival data...')
print(ED_arrival_data.head())

# counting arrivals by date
ED_arrival_counts = {}
for index,row in ED_arrival_data.iterrows():
    month = str(row['Arrival Date/Time'].month)
    day = str(row['Arrival Date/Time'].day)
    hour = str(row['Arrival Date/Time'].hour)
    date_key = month + '-' + day + '-' + hour
    if date_key not in ED_arrival_counts.keys():
        ED_arrival_counts[date_key] = 1
    else:
        ED_arrival_counts[date_key] += 1

# organizing counts by hour
import re
pattern = '.+-.+-(.+)'
ED_hour_counts = {}
for key, value in ED_arrival_counts.items():
    hour = re.findall(pattern,key)[0]
    if hour not in ED_hour_counts.keys():
        ED_hour_counts[hour] = [value]
    else:
        ED_hour_counts[hour].append(value)
        
# average arrival rate per hour for general ED patient arrivals to routine care
avg_arrivals_per_hour = [0]*24
for key, value in ED_hour_counts.items():
    idx = int(key)
    avg_arrivals_per_hour[idx] = np.mean(value)
# arrival rates in minutes for each hour
time_varying = [60/arrival for arrival in avg_arrivals_per_hour]
print("\nPreviewing time varying ED arrival rates by hour of day...")
print(time_varying)
print("mean arrival rate:",np.mean(time_varying))

Previewing ED arrival data...
    Arrival Date/Time  Month  Year Site
0 2019-01-01 01:56:00      1  2019  LFH
1 2019-01-01 05:21:00      1  2019  LFH
2 2019-01-01 07:35:00      1  2019  LFH
3 2019-01-01 10:36:00      1  2019  LFH
4 2019-01-01 11:56:00      1  2019  LFH

Previewing time varying ED arrival rates by hour of day...
[53.07692307692308, 49.285714285714285, 52.00000000000001, 48.0, 49.090909090909086, 50.0, 54.54545454545454, 42.35294117647059, 42.85714285714286, 52.00000000000001, 46.666666666666664, 48.0, 54.0, 48.33333333333333, 44.21052631578947, 52.10526315789473, 42.0, 38.4, 50.27027027027027, 42.666666666666664, 51.92307692307693, 45.599999999999994, 45.4054054054054, 43.333333333333336]
mean arrival rate: 47.75515112937713


## Simulation

In [7]:
results_df = pd.DataFrame(columns = ['DIDO', 'HS_DIDO', 'IS_DIDO', 'ICU_Times', 'Door-To-Needle', 'Triage', 'ED Eval', 
                                     'Routine Care', 'CT','Ambulance Arrival', 'Telestroke', 'Long Consent', 
                                     'Pharmacist Arrival', 'TPA Mixing', 'CTA','Total Time'])
for j in range(200):
    early_send_1_percentage = data[data.process =='early_send_1_percentage']['percentage'].item()
    early_send_2_percentage = data[data.process =='early_send_2_percentage']['percentage'].item()
    early_send_3_percentage =data[data.process =='early_send_3_percentage']['percentage'].item()
#     seed(1)
    DIDO_times = []
    hs_DIDO_times = []
    is_DIDO_times = []
    ICU_times = []
    door_to_needle = []
    Triage_times = []
    ED_eval_times = []
    routine_care_times = []
    CT_scan_times = []
    Ambulance_times = []
    Telestroke_times = []
    Long_Consent_times = []
    Pharmacist_Arrival_times = []
    Mix_TPA_times = []
    CTA_times = []

    TPA_mixing_duration_triangular = [float(x) for x in data[data.process =='TPA_mixing_duration']['triangular'].item().split(',')]
    pharmacist_arrival_duration_triangular = [float(x) for x in data[data.process =='pharmacist_arrival_duration']['triangular'].item().split(',')]

    # define environment
    env = simpy.Environment()
    # define ICU capacity
    routine_care = simpy.Resource(env, capacity=int(data[data.process =='routine_care_capacity']['count']))
    # define total number of stroke patients
    num_patients = data[data.process =='num_stroke_patients']['count']

    # general ED arrivals
    ED_arrival_time = 0
    # number of general ED patients = (num_stroke_patients*1.5*1440)/mean_ED_arrival_times
    for i in range(math.floor(int(num_patients)*1.5*1440/float(np.mean(time_varying)))):
        arrival_rate = np.random.poisson(time_varying[math.floor(ED_arrival_time%1440/60)])
        ED_arrival_time += arrival_rate
        routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
        env.process(ED_patient(env,'ED_Patient %d' % i, ED_arrival_time, routine_care_duration))

    # EMS arrivals
    EMS_arrival_time = 0
    for i in range(int(float(data[data.process =='EMS_arrivals_percentage']['percentage'])*int(num_patients))):
        EMS_screen_percentage = data[data.process =='EMS_screen_percentage']['percentage'].item()
        triage_duration = expovariate(1/float(data[data.process =='triage_duration']['mean(mins)']))
        ED_eval_duration = expovariate(1/float(data[data.process =='ED_eval_duration']['mean(mins)']))
        stroke_identified_percentage = data[data.process =='stroke_identified_percentage']['percentage'].item()
        routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
        CT_duration = expovariate(1/float(data[data.process =='CT_duration']['mean(mins)']))
        HS_percentage = data[data.process =='HS_percentage']['percentage'].item()
        ambulance_arrival_duration = expovariate(1/float(data[data.process =='ambulance_arrival_duration']['mean(mins)']))
        telestroke_duration = expovariate(1/float(data[data.process =='telestroke_duration']['mean(mins)']))
        TPA_decision_percentage = data[data.process =='TPA_decision_percentage']['percentage'].item()
        TPA_consent_percentage = data[data.process =='TPA_consent_percentage']['percentage'].item()
        TPA_long_consent_percentage = data[data.process =='TPA_long_consent_percentage']['percentage'].item()
        TPA_consent_duration = expovariate(1/float(data[data.process =='TPA_consent_duration']['mean(mins)']))
        TPA_mixing_duration = triangular(TPA_mixing_duration_triangular[0],TPA_mixing_duration_triangular[1],TPA_mixing_duration_triangular[2])
        pharmacist_arrival_duration = triangular(pharmacist_arrival_duration_triangular[0],pharmacist_arrival_duration_triangular[1],pharmacist_arrival_duration_triangular[2])
        CTA_decision_percentage = data[data.process =='CTA_decision_percentage']['percentage'].item()
        CTA_duration = expovariate(1/float(data[data.process =='CTA_duration']['mean(mins)']))
        transfer_decision_percentage = data[data.process =='transfer_decision_percentage']['percentage'].item()

        # arrival rate
        EMS_arrival_rate = np.random.poisson(float(data[data.process =='arrival_rate']['mean(mins)']))
        EMS_arrival_time += EMS_arrival_rate
        env.process(EMS_patient(env, 'EMS_Patient %d' % i, EMS_arrival_time, routine_care, EMS_screen_percentage, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage))

    # Walkin arrivals
    walkin_arrival_time = 0
    for i in range(int(float(data[data.process =='EMS_arrivals_percentage']['percentage'])*int(num_patients)),int(num_patients)):
        EMS_screen_percentage = data[data.process =='EMS_screen_percentage']['percentage'].item()
        triage_duration = expovariate(1/float(data[data.process =='triage_duration']['mean(mins)']))
        ED_eval_duration = expovariate(1/float(data[data.process =='ED_eval_duration']['mean(mins)']))
        stroke_identified_percentage = data[data.process =='stroke_identified_percentage']['percentage'].item()
        routine_care_duration = expovariate(1/float(data[data.process =='routine_care_duration']['mean(mins)']))
        CT_duration = expovariate(1/float(data[data.process =='CT_duration']['mean(mins)']))
        HS_percentage = data[data.process =='HS_percentage']['percentage'].item()
        ambulance_arrival_duration = expovariate(1/float(data[data.process =='ambulance_arrival_duration']['mean(mins)']))
        telestroke_duration = expovariate(1/float(data[data.process =='telestroke_duration']['mean(mins)']))
        TPA_decision_percentage = data[data.process =='TPA_decision_percentage']['percentage'].item()
        TPA_consent_percentage = data[data.process =='TPA_consent_percentage']['percentage'].item()
        TPA_long_consent_percentage = data[data.process =='TPA_long_consent_percentage']['percentage'].item()
        TPA_consent_duration = expovariate(1/float(data[data.process =='TPA_consent_duration']['mean(mins)']))
        TPA_mixing_duration = triangular(TPA_mixing_duration_triangular[0],TPA_mixing_duration_triangular[1],TPA_mixing_duration_triangular[2])
        pharmacist_arrival_duration = triangular(pharmacist_arrival_duration_triangular[0],pharmacist_arrival_duration_triangular[1],pharmacist_arrival_duration_triangular[2])
        CTA_decision_percentage = data[data.process =='CTA_decision_percentage']['percentage'].item()
        CTA_duration = expovariate(1/float(data[data.process =='CTA_duration']['mean(mins)']))
        transfer_decision_percentage = data[data.process =='transfer_decision_percentage']['percentage'].item()

        # arrival rate
        walkin_arrival_rate = np.random.poisson(float(data[data.process =='arrival_rate']['mean(mins)']))
        walkin_arrival_time += walkin_arrival_rate
        env.process(walkin_patient(env, 'walkin_Patient %d' % i, walkin_arrival_time, routine_care, triage_duration, ED_eval_duration, stroke_identified_percentage, routine_care_duration, CT_duration, HS_percentage, ambulance_arrival_duration, telestroke_duration, TPA_decision_percentage, TPA_consent_percentage, TPA_long_consent_percentage, TPA_consent_duration, TPA_mixing_duration, pharmacist_arrival_duration, CTA_decision_percentage, CTA_duration, transfer_decision_percentage))

    env.run(until=3000*num_patients)
#     print("\nCSC number of patients sent:",len(DIDO_times),"(%d percent)" % (len(DIDO_times)/int(num_patients)*100))
#     print("Mean DIDO time:", np.mean(DIDO_times), "minutes")
#     if early_send == 0 or 2:
#         if early_send != 2 and early_send != 3:
#             print("ICU number of patients sent:",len(ICU_times),"(%d percent)" % (len(ICU_times)/int(num_patients)*100))
#             print("Mean ICU out time", np.mean(ICU_times), "minutes")
#         print("\nMean HS DIDO time:", np.mean(hs_DIDO_times), "minutes (%d patients)" % len(hs_DIDO_times))
    print("Mean IS DIDO time:", np.mean(is_DIDO_times), "minutes (%d patients)"% len(is_DIDO_times))
#         if early_send != 2:
#             print("Mean door-to-needle time:", np.mean(door_to_needle), "minutes (%d patients)"% len(door_to_needle))
    results =  [np.mean(DIDO_times), np.mean(hs_DIDO_times), np.mean(is_DIDO_times), np.mean(ICU_times), np.mean(door_to_needle),
                np.mean(Triage_times), np.mean(ED_eval_times), np.mean(routine_care_times), np.mean(CT_scan_times), 
                np.mean(Ambulance_times), np.mean(Telestroke_times), np.mean(Long_Consent_times), np.mean(Pharmacist_Arrival_times),
                np.mean(Mix_TPA_times), np.mean(CTA_times),env.now]
    results_df.loc[j] = results
results_df.head()

Mean IS DIDO time: 101.39229771696502 minutes (274 patients)


/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean IS DIDO time: 103.550492799627 minutes (263 patients)


KeyboardInterrupt: 

In [ ]:
import statsmodels.api as sm
distributions = ['Triage', 'ED Eval', 'Routine Care', 'CT','Ambulance Arrival', 'Telestroke', 'Long Consent', 
                 'Pharmacist Arrival', 'TPA Mixing', 'CTA']
outputs = ['DIDO', 'HS_DIDO', 'IS_DIDO', 'Door-To-Needle', 'ICU_Times']
results_df.dropna(axis=0, inplace=True)
sensitivity_df = pd.DataFrame(columns=distributions, index=outputs)
for o in outputs:
    model = sm.OLS(results_df[o], results_df[distributions]).fit()
    sensitivity_df.loc[o] = model.params
sensitivity_df

In [ ]:
import matplotlib.pyplot as plt
output_toplot = 'Door-To-Needle'
plt.barh(np.arange(len(sensitivity_df.columns)), sensitivity_df.loc[output_toplot, :].values)
plt.yticks(np.arange(len(sensitivity_df.columns)), sensitivity_df.columns)
plt.title('Door to Needle Sensitivity')
plt.show()
# sensitivity_df.loc['DIDO', :].values

In [ ]:
sensitivity_df.to_csv('sensitivity.csv')